In [1]:
!pip install python-telegram-bot==13.8

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 495.2/495.2 KB 33.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 KB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 424.0/424.0 KB 48.3 MB/s eta 0:00:00
  Attempting uninstall: tornado
    Found existing installation: tornado 6.0.4
    Uninstalling tornado-6.0.4:
      Successfully uninstalled tornado-6.0.4
  Attempting uninstall: cachetools
    Found existing installation: cachetools 5.2.1
    Uninstalling cachetools-5.2.1:
      Successfully uninstalled cachetools-5.2.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires tornado~=6.0.4, but you have tornado 6.2 which is incompatible.


In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 101.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 77.8 MB/s eta 0:00:00


In [3]:
from telegram import Update
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext

In [4]:
!pip install pymorphy2
!pip install pymorphy2 stop_words annoy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 KB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 69.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=a0ecf38aacdf8d5c379fc50d105a879e372c07dbf3bbd229937af12a62c358ed
  Stored in directory: /root/.cache/pip/wheels/56/ea/58/ead137b087d9e326852a851351d1debf4ada529b6ac0ec4e8c
Successfully built docopt
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 648.0/648.0 KB 38.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for stop_words: filename=stop_words-2018.7.23-py3-none-any.whl size=32910 sha256=a6265d534c97231a07c4e7e48073176068365716829189824a7dc90

In [5]:
import string
import annoy
import pickle
import numpy as np

from tqdm.notebook import tqdm_notebook
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from gensim.models import Word2Vec, FastText
from transformers import AutoTokenizer

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
PATH_NLP = "/content/drive/MyDrive/Colab Notebooks/nlp/"
PATH_MODEL = "/content/drive/MyDrive/Colab Notebooks/nlp/models/"

In [8]:
%%time

assert True

question = None
written = False

with open(PATH_NLP + "prepared_answers.txt", "w") as fout:
    with open(PATH_NLP + "Otvety.txt", "r") as fin:
        for line in tqdm_notebook(fin):
            if line.startswith("---"):
                written = False
                continue
            if not written and question is not None:
                fout.write(question.replace("\t", " ").strip() + "\t" + line.replace("\t", " "))
                written = True
                question = None
                continue
            if not written:
                question = line.strip()
                continue

FileNotFoundError: ignored

In [9]:
morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)

In [10]:
def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [11]:
def preprocess_txt1(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i != ""]
    return spls

In [28]:
assert True

sentences = []

morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)
c = 0

with open(PATH_NLP + "Otvety.txt", "r") as fin:
    for line in tqdm_notebook(fin):
        spls = preprocess_txt(line)
        sentences.append(spls)
        c += 1
        if c > 500000:
            break

0it [00:00, ?it/s]

In [14]:
# sentences[:8]

In [31]:
# import pickle
# with open(PATH_MODEL +'sent.pkl', 'wb') as f:
#    pickle.dump(sentences, f)

In [13]:
with open(PATH_MODEL +'sent.pkl', 'rb') as f:
  sentences = pickle.load(f)

In [15]:
sentences[:8]

[[],
 [],
 ['вопрос', 'тдв', 'отдыхать', 'лично', 'советовать', 'завести'],
 ['хомячок'],
 ['мужик', 'йопарить', 'собачка', '50', 'кошка'],
 ['общение'],
 ['паучок'],
 ['пол', 'памытьbr', 'таг', 'тип', 'каво']]

In [16]:
sentences = [i for i in sentences if len(i) > 2]

In [17]:
# modelFT = FastText(sentences=sentences, min_count=1, window=5, workers=4)
# modelFT.save(PATH_MODEL + "ft_model")

In [18]:
modelFT = FastText.load(PATH_MODEL + "ft_model")

In [37]:
# ft_index = annoy.AnnoyIndex(100 ,'angular')

# index_map = {}
# counter = 0

# with open(PATH_NLP + "prepared_answers.txt", "r") as f:
#     for line in tqdm_notebook(f):
#         n_ft = 0
#         spls = line.split("\t")
#         index_map[counter] = spls[1]
#         question = preprocess_txt(spls[0])
#         vector_ft = np.zeros(100)
#         for word in question:
#             if word in modelFT.wv:
#                 vector_ft += modelFT.wv[word]
#                 n_ft += 1
#         if n_ft > 0:
#             vector_ft = vector_ft / n_ft
#         ft_index.add_item(counter, vector_ft)
            
#         counter += 1

# ft_index.build(10)
# ft_index.save(PATH_MODEL + 'speaker.ann')

In [20]:
# with open(PATH_MODEL + 'index_map.pkl', 'wb') as fp:
#    pickle.dump(index_map, fp)
with open(PATH_MODEL + 'index_map.pkl', 'rb') as f:
  index_map = pickle.load(f)

In [21]:
ft_index = annoy.AnnoyIndex(100, 'angular')
ft_index.load(PATH_MODEL + 'speaker.ann') 

True

In [22]:
ft_index.get_nns_by_vector(np.zeros(100), 2)

[59810, 61316]

In [23]:
import pandas as pd

In [24]:
df_recepies = pd.read_csv(PATH_NLP + 'all_recepies_inter.csv', sep='\t')

In [25]:
df_recepies.head()

,Unnamed: 0,name,composition,cooking_type,Инструкции,dish_type,Дата,photo,source,composition_inter
0,0,рассольник классический с перловкой и солеными...,"[{'Перловка': 0.1, 'unit': 'стак. (200 мл)'}, ...","варка,жарка",Подготовить указанные ингредиенты для приготов...,первое,05.06.2015,photo_1000menu_1.jpg,https://1000.menu/cooking/33395-rassolnik-s-pe...,"[{'product_id': 4253, 'name_source': 'Перловая..."
1,1,Суп пюре из белокочаной капусты,"[{'Капуста белокочанная': 50.0, 'unit': 'гр'},...",варка,"Необходимые ингредиенты\r\nНарезаем лук, морко...",первое,27.06.2015,photo_1000menu_2.jpg,https://1000.menu/cooking/25399-sup-pure-iz-be...,"[{'product_id': 2286, 'name_source': 'Капуста ..."
2,2,Постные щи из квашеной капусты,"[{'Капуста квашеная': 116.7, 'unit': 'гр'}, {'...","варка,жарка,тушение","Честно признаюсь, у меня не было репы на момен...",первое,12.02.2013,photo_1000menu_3.jpg,https://1000.menu/cooking/5159-postnje-shchi,"[{'product_id': 0, 'name_source': 'Капуста ква..."
3,3,Тюря- простой суп быстро и вкусно,"[{'Квас': 0.2, 'unit': 'л'}, {'Лук репчатый': ...",сырое,"\r\nНачинаем мы приготовление тюри с того, что...",первое,02.03.2011,photo_1000menu_4.jpg,https://1000.menu/cooking/5085-turya,"[{'product_id': 0, 'name_source': 'Квас', 'uni..."
4,4,Фасолевый суп из красной фасоли,"[{'Вода': 0.3, 'unit': 'л'}, {'Картошка': 0.3,...",варка,Подготовить ингредиенты. Для приготовления суп...,первое,28.01.2013,photo_1000menu_5.jpg,https://1000.menu/cooking/38765-fasolevyi-sup-...,"[{'product_id': 828, 'name_source': 'Вода', 'u..."


In [26]:
df_recepies.shape

(27884, 10)

In [27]:
data = df_recepies['Инструкции']

In [28]:
data.head()

0    Подготовить указанные ингредиенты для приготов...
1    Необходимые ингредиенты\r\nНарезаем лук, морко...
2    Честно признаюсь, у меня не было репы на момен...
3    \r\nНачинаем мы приготовление тюри с того, что...
4    Подготовить ингредиенты. Для приготовления суп...
Name: Инструкции, dtype: object

In [29]:
import re
from sklearn.model_selection import train_test_split

def build_text_files(data_json, dest_path):
    f = open(dest_path, 'w')
    data = ''
    for texts in data_json:
        summary = str(texts).strip()
        summary = re.sub(r"\s", " ", summary)
        data += summary + "  "
    f.write(data)

In [30]:
data.shape

(27884,)

In [31]:
data[8]

'Чтобы приготовить постный гороховый суп подготовьте горох, картофель, морковь, лук, растительное масло (поскольку блюдо постное, то исключаем сливочное или топленое масло). Также потребуется немного соли, пряностей и свежей зелени.\r\nЧтобы горох идеально разварился и суп получился вкусным, есть небольшой секрет начала приготовления постного горохового супа.Залейте сухой горох (замачивать не нужно) небольшим количеством КИПЯЩЕЙ воды. Добавьте чуть-чуть соли, на кончике ножа молотой корицы, лавровый лист и чуть чуть растительного масла. Доведите до кипения и варите до разбухания гороха, затем воду можно долить до нужного количества. Так не придется стоять у плиты, постоянно снимая пену\r\nПока варится горох, почистите морковь и репчатый лук. Морковь натрите на крупной терке, лук мелко нарежьте кусочками произвольной формы. Разогрейте растительное масло с пряностями (карри, куркума, черный молотый перец, паприка), добавьте измельченные лук и морковь. Спассеруйте овощи до мягкости.\r\nКа

In [32]:
# with open(PATH_MODEL + 'data.pkl', 'wb') as f:
#    pickle.dump(data, f)
with open(PATH_MODEL + 'data.pkl', 'rb') as f:
  data = pickle.load(f)

In [33]:
train, test = train_test_split(data, test_size=0.2)

build_text_files(train,'train_dataset.txt')
build_text_files(test,'test_dataset.txt')

In [34]:
tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [35]:
train_path = 'train_dataset.txt'
test_path = 'test_dataset.txt'

In [36]:
from transformers import TextDataset, DataCollatorForLanguageModeling

def load_dataset(train_path, test_path, tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)

    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset, test_dataset, data_collator

train_dataset, test_dataset, data_collator = load_dataset(train_path, test_path, tokenizer)

/usr/local/lib/python3.8/dist-packages/transformers/data/datasets/language_modeling.py:54: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [37]:
from transformers import Trainer, TrainingArguments, AutoModelForCausalLM

In [38]:
model = AutoModelForCausalLM.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")

In [39]:
training_args = TrainingArguments(
    output_dir= PATH_MODEL + "gpt2_shop",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    eval_steps = 400,
    save_steps=800,
    warmup_steps=500,
    )

In [61]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

In [62]:
# trainer.train()

In [40]:
# tokenizer.save_pretrained(PATH_MODEL +'gpt_chf')
model.save_pretrained(PATH_MODEL +'model_gpt_chf')

In [41]:
from transformers import AutoTokenizer

In [42]:
tokenizer = AutoTokenizer.from_pretrained(PATH_MODEL +'gpt_chf')

In [43]:
from transformers import Trainer, TrainingArguments, AutoModelForCausalLM

In [44]:
model = AutoModelForCausalLM.from_pretrained(PATH_MODEL +'model_gpt_chf')

In [45]:
prefix = "томаты и сыр"

In [46]:
tokens = tokenizer(prefix, return_tensors='pt')

In [47]:
size = tokens['input_ids'].shape[1]
output = model.generate( 
    **tokens, 
    do_sample=False, 
    max_length=size+100, 
    repetition_penalty=5., 
    temperature=1.,
    num_beams=10,
)

decoded = tokenizer.decode(output[0])
result = decoded[len(prefix):]
print(prefix + result)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


томаты и сырники.

— А что это такое? — спросил я, когда мы сели за стол.

— Я не знаю, — пожала плечами она. — Но мне кажется, в этом есть какой-то смысл.

— В каком смысле?

— Во всех смыслах. В том, как ты относишься к тому, чтобы заниматься сексом с кем-то другим. Это ведь так здорово — быть рядом с тем, кто тебе нравится.


In [48]:
def gpt_generate(prefix):
  tokens = tokenizer(prefix, return_tensors='pt')
  size = tokens['input_ids'].shape[1]
  output = model.generate( 
      **tokens, 
      do_sample=False, 
      max_length=size+100, 
      repetition_penalty=5., 
      temperature=1.,
      num_beams=10,
  )

  decoded = tokenizer.decode(output[0])
  result = decoded[len(prefix):]
  return prefix + result

In [49]:
prefix = "маскарпоне и творог"

In [50]:
gpt_generate(prefix)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'маскарпоне и творог с изюмом.  В качестве начинки можно использовать любые сухофрукты: курагу, чернослив, абрикосы, яблоки, груши, сливы, вишни, смородину, ежевику, клюкву, бруснику, чернику, малины, клубнику, малину, черемуху, шиповник, землянику, крыжовник, облепиху, крапиву, щавель, одуванчик, мяту,'

# Обучим модель классифицировать интенты

In [52]:
# data_new = data.apply(lambda x: preprocess_txt(str(x)))

In [53]:
# data_new[:8]

In [54]:
# with open(PATH_MODEL +'data_new.pkl', 'wb') as f:
#    pickle.dump(data, f)
with open(PATH_MODEL +'data_new.pkl', 'rb') as f:
  data_new = pickle.load(f)

In [55]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

vectorizer = CountVectorizer(ngram_range=(1, 2))

In [56]:
idxs = set(np.random.randint(0, len(index_map), len(data)))

In [57]:
negative_texts = [" ".join(preprocess_txt(index_map[i])) for i in idxs]
positive_texts = [" ".join(val) for val in data_new.values]

In [58]:
negative_texts[8]

'поехать какоенибудь интересный место оттянуться полный намекнуть толком говорить'

In [59]:
positive_texts[8]

'Ч т о б ы   п р и г о т о в и т ь   п о с т н ы й   г о р о х о в ы й   с у п   п о д г о т о в ь т е   г о р о х ,   к а р т о ф е л ь ,   м о р к о в ь ,   л у к ,   р а с т и т е л ь н о е   м а с л о   ( п о с к о л ь к у   б л ю д о   п о с т н о е ,   т о   и с к л ю ч а е м   с л и в о ч н о е   и л и   т о п л е н о е   м а с л о ) .   Т а к ж е   п о т р е б у е т с я   н е м н о г о   с о л и ,   п р я н о с т е й   и   с в е ж е й   з е л е н и . \r \n Ч т о б ы   г о р о х   и д е а л ь н о   р а з в а р и л с я   и   с у п   п о л у ч и л с я   в к у с н ы м ,   е с т ь   н е б о л ь ш о й   с е к р е т   н а ч а л а   п р и г о т о в л е н и я   п о с т н о г о   г о р о х о в о г о   с у п а . З а л е й т е   с у х о й   г о р о х   ( з а м а ч и в а т ь   н е   н у ж н о )   н е б о л ь ш и м   к о л и ч е с т в о м   К И П Я Щ Е Й   в о д ы .   Д о б а в ь т е   ч у т ь - ч у т ь   с о л и ,   н а   к о н ч и к е   н о ж а   м о л о т о й   к о р и ц ы ,   л а в р о в

In [60]:
dataset = negative_texts + positive_texts
labels = np.zeros(len(dataset))
labels[len(negative_texts):] = np.ones(len(positive_texts))

In [61]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(dataset, labels, test_size=0.2, stratify=labels, random_state=42)

In [62]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 18.1 MB/s eta 0:00:00


In [65]:
from catboost import CatBoostClassifier
cat = CatBoostClassifier()
x_train_vec = vectorizer.fit_transform(X_train)
x_test_vec = vectorizer.transform(X_test)

# cat_new = cat.fit(x_train_vec, y_train)

In [68]:
input_txt = preprocess_txt('взять помидор подготовить лук томатная паста')
vect = vectorizer.transform([" ".join(input_txt)])
prediction = cat_new.predict_proba(vect)
print(prediction)

[[0.26280511 0.73719489]]


In [69]:
from sklearn.metrics import accuracy_score
accuracy_score(y_true=y_test, y_pred=cat_new.predict(x_test_vec))

0.5091991341991342

In [88]:
# with open(PATH_MODEL +'cat_new.pkl', 'wb') as f:
#    pickle.dump(cat_new, f)

In [67]:
with open(PATH_MODEL +'cat_new.pkl', 'rb') as f:
   cat_new = pickle.load(f)

In [70]:
def embed_txt(txt, idfs, midf):
    n_ft = 0
    vector_ft = np.zeros(100)
    for word in txt:
        if word in modelFT.wv:
            vector_ft += modelFT.wv[word] * 1
            n_ft += 1
    return vector_ft / n_ft

# Подключаем Бота

In [71]:
import os
import datetime
import logging
from telegram import Update
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext

In [72]:
BOT_API = '5869913820:AAHGvL0LGWIIiZN_J2uDY8AHdvTKy5I8lZU'
updater = Updater(BOT_API, use_context=True)
dispatcher = updater.dispatcher

In [73]:
key_time = ['время','час']
key_thanks = ['спасибо']
def check_in_list(text, key_list):
    for w in text:
      if w in key_list:
        flag = True
        break
      else:
        flag = False
    return flag

In [74]:
def startCommand(update: Update, context: CallbackContext):
    update.message.reply_text('Салют! Я могу узнать который час или найти рецепт')

def textMessage(update: Update, context: CallbackContext):
    input_txt = preprocess_txt(update.message.text)
    input_txt1 = preprocess_txt1(update.message.text)
    vect = vectorizer.transform([" ".join(input_txt)])
    prediction = cat_new.predict(vect)
    
    if check_in_list(input_txt1, key_thanks): 
            
        update.message.reply_text('Всегда рад быть полезным!')
        return
    
    if check_in_list(input_txt1, key_time): 
      
        now = datetime.datetime.now()       
        update.message.reply_text(now.strftime("%d-%m-%Y %H:%M"))
        return

    if prediction == 0:
        vect_ft = embed_txt(input_txt, {}, 1)
        ft_index_val, distances = ft_index.get_nns_by_vector(vect_ft, 1, include_distances=True)
        if distances[0] > 0.45:
          print(distances[0])
          update.message.reply_text("Уточните, пожалуйста, вопрос")
          return
        update.message.reply_text(index_map[ft_index_val[0]])
        return

    update.message.reply_text(gpt_generate(update.message.text))

In [ ]:
dispatcher.add_handler(CommandHandler("start", startCommand))
dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, textMessage))

updater.start_polling()
updater.idle()

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
